In [40]:
#get sample index of "{"
import math
from collections import Counter, defaultdict
def get_info_and_exec(sample: str) -> dict:
    try:
        info = sample[:sample.index("{")].strip()
        task_id, solution_id, base_plus = info.split(" ")
    except:
        task_id, solution_id, base_plus = sample.strip().split(" ")
        sample = "{}\n"
    try:
        execution = eval(sample[sample.index("{"):].strip())
    except:
        try:
            execution = sample[sample.index("{"):].strip().replace("-inf", "\'minus_ifty\'").replace("inf", "\'ifty\'")
            execution = eval(execution)
            for e in execution:
                if execution[e] == "minus_ifty":
                    execution[e] = float("-inf")
                elif execution[e] == "ifty":
                    execution[e] = float("inf")
            if task_id in ["758", "237"] and solution_id in ["55", "136", "149"]:
                print(sample)
        except:
            print(sample)
            assert task_id in ["758", "237"]
            assert solution_id in ["55", "136", "149"]
            if task_id == "758":
                if solution_id == 136:
                    execution = {0: defaultdict(int, {(1, 3): 2, (5, 7): 2, (13, 15, 17): 1, (9, 11): 1}), 
                                 1: defaultdict(int, {('green', 'orange'): 2, ('black',): 1, ('white',): 1}), 
                                 2: defaultdict(int, {(10, 20, 30, 40): 1, (60, 70, 50, 50): 1, (90, 100, 200): 1}), 
                                 3: defaultdict(int, {('john',): 1})}
                elif solution_id == 149:
                    execution = {0: Counter({(1, 3): 2, (5, 7): 2, (13, 15, 17): 1, (9, 11): 1}), 
                                 1: Counter({('green', 'orange'): 2, ('black',): 1, ('white',): 1}), 
                                 2: Counter({(10, 20, 30, 40): 1, (50, 50, 60, 70): 1, (90, 100, 200): 1}), 
                                 3: Counter({('john',): 1})}
            elif task_id == "237":
                execution = {0: Counter({(3, 1): 1, (1, 3): 1, (2, 5): 1, (5, 2): 1, (6, 3): 1}), 
                             1: Counter({(4, 2): 1, (2, 4): 1, (3, 6): 1, (6, 3): 1, (7, 4): 1}), 
                             2: Counter({(13, 2): 1, (11, 23): 1, (12, 25): 1, (25, 12): 1, (16, 23): 1})}
    return {"task_id": task_id, "solution_id": int(solution_id), "base_plus": base_plus, "execution": execution}

In [41]:
exec_outputs_dir = "/mnt/scratch-artemis/haausing/code_reranking/evalplus_outputs/mbpp/deepseek-coder-7b-instruct-v1.5_temp_0.8/exec_outputs_base_only.txt"
#read lines of exec_outputs_dir
with open(exec_outputs_dir, "r") as f:
    exec_outputs = [e for e in f.readlines() if e != "\n"]
exec_outputs = "".join(exec_outputs).split("Mbpp/")[1:]
exec_outputs[-1] = exec_outputs[-1].split("\n")[0]
#exec_outputs = [e for e in exec_outputs if "Mbpp" in e]
# count the number of "Mbpp" in exec_outputs
#from tqdm import tqdm
#for i,line in tqdm(enumerate(exec_outputs)):
#    if line.count("Mbpp") > 1:
#        execs = ["Mbpp"+e for e in line.split("Mbpp") if e != '']
#        exec_outputs[i] = execs[0]
#        exec_outputs.extend(execs[1:])
#exec_outputs = [get_info_and_exec(e) for e in exec_outputs]

In [45]:
new_exec_outputs = [get_info_and_exec(e) for e in exec_outputs]

758 136 base {0: defaultdict(<class 'int'>, {(1, 3): 2, (5, 7): 2, (13, 15, 17): 1, (9, 11): 1}), 1: defaultdict(<class 'int'>, {('green', 'orange'): 2, ('black',): 1, ('white',): 1}), 2: defaultdict(<class 'int'>, {(10, 20, 30, 40): 1, (60, 70, 50, 50): 1, (90, 100, 200): 1}), 3: defaultdict(<class 'int'>, {('john',): 1})}



In [24]:
# sort by task_id, solution_id, base_plus
new_exec_outputs = sorted(new_exec_outputs, key = lambda x: (int(x["task_id"]), x["solution_id"], x["base_plus"]))

In [56]:
import pickle
import json

work_dir = "/mnt/scratch-artemis/haausing/code_reranking/evalplus_outputs"
dataset = "humaneval"
#gen_dir = "code-llama-7b-instruct_temp_1.6"
gen_dir = "deepseek-coder-7b-instruct-v1.5_temp_0.8"

with open(f"{work_dir}/{dataset}/{gen_dir}/exec_outputs_v2.pkl", "rb") as f:
    exec_outputs = pickle.load(f)
print("exec_outputs loaded")

# load eval_results
with open(f"{work_dir}/{dataset}/{gen_dir}/eval_results.json", "r") as f:
    eval_results = json.load(f)
for task_id in eval_results["eval"]:
    eval_results["eval"][task_id] = sorted(eval_results["eval"][task_id], key=lambda x: int(x["solution_id"]))
print("eval_results loaded")

exec_outputs loaded
eval_results loaded


In [57]:
# load pickle file of log probs
#with open(f"{work_dir}/{dataset}/{gen_dir}/logprobs_full.pkl", "rb") as f:
#    log_probs = pickle.load(f)
#print("logprobs loaded")

In [58]:
import os
import time

from evalplus.data import (
    get_human_eval_plus,
    get_human_eval_plus_hash,
    get_mbpp_plus,
    get_mbpp_plus_hash,
    load_solutions,
)

from evalplus.eval._special_oracle import (
    MBPP_OUTPUT_NOT_NONE_TASKS,
    MBPP_OUTPUT_SET_EQ_TASKS,
    _poly,
)

from evalplus.data.utils import CACHE_DIR

from evalplus.gen.util import trusted_exec

def get_groundtruth(problems, hashcode, tasks_only_output_not_none):
    cache_file = os.path.join(CACHE_DIR, f"{hashcode}.pkl")
    if os.path.exists(cache_file):
        #print(f"Load from ground-truth from {cache_file}")
        with open(cache_file, "rb") as f:
            return pickle.load(f)

    os.makedirs(CACHE_DIR, exist_ok=True)
    #print("Computing expected output...")
    tbegin = time.time()
    expected_output = {}
    for task_id, problem in problems.items():
        oracle = {}
        oracle["base"], oracle["base_time"] = trusted_exec(
            problem["prompt"] + problem["canonical_solution"],
            problem["base_input"],
            problem["entry_point"],
            record_time=True,
            output_not_none=problem["entry_point"] in tasks_only_output_not_none,
        )

        oracle["plus"], oracle["plus_time"] = trusted_exec(
            problem["prompt"] + problem["canonical_solution"],
            problem["plus_input"],
            problem["entry_point"],
            record_time=True,
            output_not_none=problem["entry_point"] in tasks_only_output_not_none,
        )
        expected_output[task_id] = oracle
    #print(f"Expected outputs computed in {time.time() - tbegin:.2f}s")

    with open(cache_file, "wb") as f:
        pickle.dump(expected_output, f)

    return expected_output

# load problems
if dataset == "mbpp":
    problems = get_mbpp_plus()
    dataset_hash = get_mbpp_plus_hash()
    expected_output = get_groundtruth(
        problems,
        dataset_hash,
        MBPP_OUTPUT_NOT_NONE_TASKS,
    )
elif dataset == "humaneval":
    problems = get_human_eval_plus()
    dataset_hash = get_human_eval_plus_hash()
    expected_output = get_groundtruth(
        problems,
        dataset_hash,
        []
    )
else:
    raise ValueError("Invalid dataset")


In [59]:
#src_idx, sample_idx, result_unit_test_1, result_unit_test_2,  ..., result_unit_test_X
final_outputs = []
for task_id in exec_outputs:
    #if task_id in ["Mbpp/6", "Mbpp/7", "Mbpp/8", "Mbpp/9"]:
        #continue
    for solution_id in range(200):
        try:
            assert solution_id == int(eval_results["eval"][task_id][solution_id]["solution_id"])
            p_name = task_id.replace("/", "_")
            final_outputs.append(
                {
                    "src_idx": int(task_id.split("/")[-1]),
                    "sample_idx": solution_id,
                    #"log_prob": sum(log_probs[p_name][solution_id]),
                    #"norm_log_prob": sum(log_probs[p_name][solution_id])/len(log_probs[p_name][solution_id]),
                    "result_unit_test_1": exec_outputs[task_id][solution_id]["base"][0] 
                        if 0 in exec_outputs[task_id][solution_id]["base"] else "failed: no output",
                    "result_unit_test_2": exec_outputs[task_id][solution_id]["base"][1] 
                        if 1 in exec_outputs[task_id][solution_id]["base"] else "failed: no output",
                    "result_unit_test_3": exec_outputs[task_id][solution_id]["base"][2] 
                        if 2 in exec_outputs[task_id][solution_id]["base"] else "failed: no output",
                    "ground_truth_test_1": expected_output[task_id]["base"][0]
                        if len(expected_output[task_id]["base"])>0 else "failed: no output",
                    "ground_truth_test_2": expected_output[task_id]["base"][1]
                        if len(expected_output[task_id]["base"])>1 else "failed: no output",
                    "ground_truth_test_3": expected_output[task_id]["base"][2]
                        if len(expected_output[task_id]["base"])>2 else "failed: no output",
                    #"ground_truth_test_1": eval_results["eval"][task_id][solution_id]["base_details"][0]==1 
                    #    if len(eval_results["eval"][task_id][solution_id]["base_details"])>0 else False,
                    #"ground_truth_test_2": eval_results["eval"][task_id][solution_id]["base_details"][1]==1 
                    #    if len(eval_results["eval"][task_id][solution_id]["base_details"])>1 else False,
                    #"ground_truth_test_3": eval_results["eval"][task_id][solution_id]["base_details"][2]==1 
                    #    if len(eval_results["eval"][task_id][solution_id]["base_details"])>2 else False,
                }
            )
        except:
            print(task_id, solution_id)
            print(exec_outputs[task_id][solution_id]["base"])
            print(expected_output[task_id]["base"])
        


In [60]:
#transform final_outputs to pandas dataframe
import pandas as pd

# Convert the list of dictionaries to a pandas DataFrame
df_final_outputs = pd.DataFrame(final_outputs)


In [61]:
#save df_final_outputs to pickle
df_final_outputs.to_pickle(f"{work_dir}/{dataset}_{gen_dir}_final_outputs.pkl")



In [62]:
gen_dir

'deepseek-coder-7b-instruct-v1.5_temp_0.8'

In [33]:
# Count how many result_unit_test_4 are not None across different src_idx
result_unit_test_4_count = df_final_outputs.groupby('src_idx')['result_unit_test_4'].apply(lambda x: x.notnull().sum())
# print the src_idx where result_unit_test_4_count > 0
#print(result_unit_test_4_count[result_unit_test_4_count > 0])

In [66]:
from evalplus.data import get_mbpp_plus
problems = get_mbpp_plus()
MBPP_OUTPUT_NOT_NONE_TASKS = ["check_str", "text_match_three", "text_starta_endb"]
for task_id in problems:
    if problems[task_id]["entry_point"] in MBPP_OUTPUT_NOT_NONE_TASKS:
        print(task_id, problems[task_id]["entry_point"])

Mbpp/737 check_str
Mbpp/787 text_match_three
Mbpp/794 text_starta_endb


In [ ]:
import numpy as np
def is_floats(x) -> bool:
    # check if it is float; List[float]; Tuple[float]
    if isinstance(x, float):
        return True
    if isinstance(x, (list, tuple)):
        return all(isinstance(i, float) for i in x)
    if isinstance(x, np.ndarray):
        return x.dtype == np.float64 or x.dtype == np.float32
    return False

def utility(output_hyp, output_ref, src_idx, sample_idx):
    if output_hyp is None or output_ref is None:
        return False
    if output_hyp.startswith("failed"):
        return False
    exact_match = output_hyp == output_ref
    if is_floats(output_hyp) or is_floats(output_ref):
        if not exact_match:
            exact_match = np.isclose(output_hyp, output_ref, atol=1e-6)
    return exact_match

In [74]:
MBPP_OUTPUT_SET_EQ_TASKS

['similar_elements',
 'find_char_long',
 'common_in_nested_lists',
 'extract_singly',
 'larg_nnum',
 'intersection_array',
 'find_dissimilar',
 'Diff']

In [77]:
for task_id in problems:
    if problems[task_id]["entry_point"] in MBPP_OUTPUT_SET_EQ_TASKS:
        print(task_id.split("/")[-1])


2
7
111
140
232
249
579
769


In [79]:
for task_id in problems:
    if problems[task_id]["entry_point"] in MBPP_OUTPUT_SET_EQ_TASKS:
        print(task_id.split("/")[-1])

2
7
111
140
232
249
579
769


In [80]:
for task_id in problems:
    if problems[task_id]["entry_point"] in MBPP_OUTPUT_NOT_NONE_TASKS:
        print(task_id.split("/")[-1])

737
787
794


In [82]:
for task_id in problems:
    if problems[task_id]["entry_point"] == "sum_div":
        print(task_id.split("/")[-1])

295


In [ ]:
import numpy as np

MBPP_OUTPUT_SET_EQ_TASK_IDS = [2,7,111,140,232,249,579,769]
MBPP_OUTPUT_NOT_NONE_TASK_IDS = [737,787,794]

def is_floats(x) -> bool:
    # check if it is float; List[float]; Tuple[float]
    if isinstance(x, float):
        return True
    if isinstance(x, (list, tuple)):
        return all(isinstance(i, float) for i in x)
    if isinstance(x, np.ndarray):
        return x.dtype == np.float64 or x.dtype == np.float32
    return False

def utility(hyp_ut, ref_ut, src_idx, dataset, inp=None, atol=0):
    
    if hyp_ut.startswith("failed:") or ref_ut.startswith("failed:"):
        return False

    exact_match = hyp_ut == ref_ut

    # ================================================ #
    # ============== special oracles ================= #
    if dataset == "mbpp":
        if 164 == src_idx:  # Mbpp/164 special oracle
            exact_match = exact_match or True
        elif 295 == src_idx:  # Mbpp/295 special oracle
            exact_match = exact_match or hyp_ut == 0 or ref_ut == 0
        elif src_idx in MBPP_OUTPUT_SET_EQ_TASK_IDS:
            exact_match = set(hyp_ut) == set(ref_ut)
        elif src_idx in MBPP_OUTPUT_NOT_NONE_TASK_IDS:
            # exp is True  if not None
            #        False if None
            if isinstance(hyp_ut, bool):
                hyp_ut = hyp_ut is not None
            if isinstance(ref_ut, bool):
                ref_ut = ref_ut is not None
            exact_match = hyp_ut == ref_ut

    if dataset == "humaneval":
        raise NotImplementedError # TODO: implement humaneval special entry points
        if "find_zero" == entry_point:
            hyp_ut = _poly(*inp, hyp_ut) <= atol
            ref_ut = _poly(*inp, ref_ut) <= atol
            exact_match = hyp_ut == ref_ut
    # ============== special oracles ================= #
    # ================================================ #

    if atol == 0 and (is_floats(ref_ut) or is_floats(hyp_ut)):
        atol = 1e-6  # enforce atol for float comparison
    if not exact_match and atol != 0:
        # explicitly set rtol=1e-07
        # to match `np.testing.assert_allclose`'s default values
        exact_match =  np.allclose(hyp_ut, ref_ut, rtol=1e-07, atol=atol)
    
    return exact_match